In [5]:
import os
import anthropic
from IPython.display import Markdown, display
from docx import Document

import os
from pathlib import Path

_root = Path.cwd()
_utils_dir = _root.parent.parent.as_posix()  #'/home/zjc1002/Mounts/code/MyModules/utils
os.sys.path.append(_utils_dir)
from utils.misc import LoadCFG
config_path = (_root / "config.yaml").as_posix()
cfg = LoadCFG(config_path, base_dir=_root.as_posix(), return_namespace=False).load()


#Anthropic API config params
api_key = cfg['api_key'] #my api key
model = cfg['model'] #name of anthropic model to use 
max_tokens = cfg['max_tokens'] #max tokens in anthropic model response 
temperature = cfg['temperature'] #how diverse do you want to the responses to be?

def load_docx_text(file_path):
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

File DOES exist:
	 /home/zjc1002/Mounts/code/MyModules/genai/claude3_ops/config.yaml


In [6]:
#initaite client 
client = anthropic.Anthropic(api_key=api_key)

### Define inputs to prompt to use in tuning coverletter 

In [8]:

#get coverletter text
coverletter_path = "/home/zjc1002/Mounts/coverletter/Zachary Carideo CoverLetter 2024.docx"
resume_path = "/home/zjc1002/Mounts/resume/Zach Carideo Resume 2024.docx"

#coverletter
coverletter = load_docx_text(coverletter_path)

#resume
resume = load_docx_text(resume_path)

#company 
company_name = 'Microsoft'

#job description 
job_description = f"""The Industry Solutions Engineering (ISE) team is a global engineering organization that works directly with customers looking to leverage the latest technologies to address their toughest challenges. We work closely with our customers’ engineers to jointly develop code for cloud-based solutions that can accelerate their organization. We work in collaboration with Microsoft product teams, partners, and open-source communities to empower our customers to do more with the cloud. We pride ourselves in making contributions to open source and making our platforms easier to use. 

We develop solutions side-by-side with our customers through collaborative innovation to solve their challenges. This work involves the development of broadly applicable, high-impact solution patterns and open-source software assets that contribute to the Microsoft platform. In this role, you will be working with engineers from your team and our customers’ teams to apply your skills, perspectives, and creativity to grow as engineers and help solve our customers’ toughest challenges. 

We are hiring a Senior Data Scientist with deep experience in data management and expertise in developing statistical techniques to analyze data and find patterns. As part of our team, you will be working side-by-side with high-impact engineers and strategic customers to solve complex problems. You will communicate trends and innovative solutions to stakeholders. You will work cross-functionally with several teams including crews, product teams, and program management to deploy business solutions.  

Our team prides itself on embracing a growth mindset, inspiring excellence, and encouraging everyone to share their unique viewpoints and be their authentic selves. Join us and help create life-changing innovations that impact billions around the world! """

#job skills 
job_skills = f"""
- Understanding Buisness Impact: Lead data-driven projects with business acumen and data science expertise. 
- Modeling & Statistical Analysis: Apply machine learning solutions and algorithms to achieve objectives, prepare and evaluate data, and communicate findings and risks. Writes scripts in various languages and understands Microsoft AI and ML tools. Designs experiments and operationalizes models at scale. Coaches engineers on best practices. 
- Model Evaluation: Understand relationship between selected models and business objectives. Ensures clear linkage between selected models and desired business objectives. Defines and designs feedback and evaluation methods. Coaches and mentors less experienced engineers as needed. Presents results and findings to senior customer stakeholders. 
- Industry Research & Lead Generation: Provide feedback, coaching, and support to engineering team and other teams based on business knowledge, technical expertise, and industry trends. 
- Coding and Debugging: Demonstrates effective coding and debugging skills across multiple features/solutions. 
- Business Managment: Drive business value by collaborating with stakeholders and improving solutions. 
- Customer/Partner Orientation: Deliver customer-oriented solutions and builds trust with Microsoft products. 
- Master's Degree in Data Science, Mathematics, Statistics, Econometrics, Economics, Operations Research, Computer Science, or related field AND 5+ years data-science experience (e.g., managing structured and unstructured data, applying statistical techniques and reporting results)  
- Demonstrated ability to work with customers and collaborate across company boundaries 
- 2+ years customer-facing, project-delivery experience, professional services, and/or consulting experience.  
- Ability to obtain and maintain United States Security Clearance. 
- Experience working as part of geographically dispersed, diverse, and virtual teams 
- Enjoy travel and are comfortable with travel up to 25%  
"""


### Initial Function to ask Claude to generate the most appriopriate prompt for updating a coverletter 

In [9]:
def build_coverletter_prompt( resume:str, coverletter: str, job_description: str, job_skills: str):
#get claude to provide potential prompts to use 

    prompt_ = f"""You are a data scientist who is updating a [coverletter] to better align with a specific [job description] and [job skills].  
    You recently came across Claude3 Opus, which can answer anything with the right prompt. You understand Claude's limitations and how to explain the prompt in detail.

    Given the data scientists [resume], and there existing [coverletter], create a list of very detailed Claude prompts(don't ask questions) that update the [coverletter] 
    to better reflect the [job description] and [job skills] using the skills and experiences from the data scientists [resume].     
    Prompts should generate a new coverletter that has been customized to reflect the [job description] using the skills and experiences provided in the provided [resume].
    Understand each prompt and insert placeholders where you think the user needs to input their data to get the prompt working to its full potential.

    coverletter: [{coverletter}]
    job description: [{job_description}]
    job skills: [{job_skills}]
    resume: {resume}

    """

    messages = [{'role': 'user', 'content': prompt_}]

    return messages 

def get_completion(client, messages, model=None, max_tokens=3000):

    response = client.messages.create(
            model=model,
            max_tokens=max_tokens,
            messages=messages
        )

    return response.content[0].text

In [ ]:
#generate multiple prompts to explore when requesting new coverletter be created
output = get_completion(client
                        , model = model 
                        , max_tokens = 3000
                        , messages = build_coverletter_prompt(resume,coverletter, job_description, job_skills )
)

print(output)

#### Output from 1st Prompt is used to generate final prompt to use in requesting coverletter be updated to reflect job description and skills. 

In [10]:
#test the prompts 
p1 = """
coverletter =  [{coverletter}]
job description =  [{job_description}]
job skills =  [{job_skills}]
resume = [{resume}] 

Claude, given the provided cover letter, job description, job skills, and resume, rewrite the cover letter to include:
- An opening paragraph that grabs attention by directly addressing the candidate's fit for the Senior Data Scientist role based on their experience leading data-driven projects with business acumen and data science expertise.
- Examples of the candidate's experience with modeling and statistical analysis, including applying machine learning solutions, preparing and evaluating data, and communicating findings and risks. Use specific projects from the resume.
- Discussion of the candidate's ability to understand the relationship between selected models and business objectives, and to design feedback and evaluation methods. Tie this to the job requirements.
- Mention of the candidate's coding and debugging skills across multiple features/solutions, as evidenced by their experience with various programming languages and tools listed in the resume.
- A closing paragraph that reiterates the candidate's customer-oriented approach, ability to drive business value through collaboration, and enthusiasm for the role.
"""

p2 = """

Claude, given the provided cover letter, job description, job skills, and resume, rewrite the cover letter to include:
- An opening paragraph that grabs attention by directly addressing the candidate's fit for the Senior Data Scientist role based on their experience leading data-driven projects with business acumen and data science expertise.
- Examples of the candidate's experience with modeling and statistical analysis, including applying machine learning solutions, preparing and evaluating data, and communicating findings and risks. Use specific projects from the resume.
- Discussion of the candidate's ability to understand the relationship between selected models and business objectives, and to design feedback and evaluation methods. Tie this to the job requirements.
- Mention of the candidate's coding and debugging skills across multiple features/solutions, as evidenced by their experience with various programming languages and tools listed in the resume.
- A closing paragraph that reiterates the candidate's customer-oriented approach, ability to drive business value through collaboration, and enthusiasm for the role.

The coverletter, job description, job skills, and resume are provided below. 

coverletter =  [{coverletter}]
job description =  [{job_description}]
job skills =  [{job_skills}]
resume = [{resume}] 
"""

def build_coverletter(prompt:str,  resume:str, coverletter: str, job_description: str, job_skills: str):

    #complete prompt with required inputs 
    prompt_ = prompt.format(coverletter=coverletter
                            , job_description=job_description
                            , job_skills = job_skills
                            , resume=resume) 

    

    messages = [{'role': 'user', 'content': prompt_}]

    return messages 

output = get_completion(client
                        , model = model 
                        , max_tokens = 3000
                        , messages = build_coverletter(p1,resume,coverletter, job_description, job_skills )
)

print(output)             

Dear Hiring Manager,

With over 10 years of experience leading data-driven projects that leverage my business acumen and data science expertise, I am confident in my ability to excel as a Senior Data Scientist on the Industry Solutions Engineering team at Microsoft. My background in managing quantitative teams focused on developing enterprise AI/ML solutions, along with my proven success in cultivating high-performing teams, make me an ideal candidate for this role.

Throughout my career, I have applied machine learning solutions and algorithms to achieve business objectives, prepare and evaluate data, and communicate findings and risks. For example, at Wells Fargo, I managed the development of a zero-shot information retrieval framework using Retrieval Augmented Generation, which is expected to capture over $10 million in FTE resources annually. Additionally, I led a team in developing seq2seq models using Keras and PyTorch to identify, monitor, and quantify emerging non-financial ris

### Cover Letter Geneation Prompts provided by Claude

Here are some detailed prompts for Claude to update your cover letter to better reflect the job description and skills using your resume:

1. [coverletter], [job description], [job skills], [resume]
Claude, given the provided cover letter, job description, job skills, and resume, rewrite the cover letter to:
- Highlight the candidate's experience managing data science teams focused on developing enterprise AI/ML solutions, including LLM frameworks. Provide specific examples from the resume.
- Emphasize the candidate's ability to cultivate high-performing teams and collaborate with leaders across the organization to identify opportunities for driving value through AI/ML initiatives. 
- Showcase the candidate's technical expertise in credit risk modeling, Python, SAS, SparkML, Pytorch, Keras, H2o, HuggingFace, and Langchain. Explain how this aligns with the job requirements.
- Discuss the candidate's experience working with customers and delivering customer-oriented solutions, as mentioned in the job description.
- Highlight the candidate's advanced degrees in relevant fields and 10+ years of data science experience.
- Ensure the tone matches the job description's emphasis on collaboration, innovation, and driving business value.

2. [coverletter], [job description], [job skills], [resume] 
Claude, update the provided cover letter to include the following:
- An opening paragraph that grabs attention by directly addressing the candidate's fit for the Senior Data Scientist role based on their experience leading data-driven projects with business acumen and data science expertise.
- Examples of the candidate's experience with modeling and statistical analysis, including applying machine learning solutions, preparing and evaluating data, and communicating findings and risks. Use specific projects from the resume.
- Discussion of the candidate's ability to understand the relationship between selected models and business objectives, and to design feedback and evaluation methods. Tie this to the job requirements.
- Mention of the candidate's coding and debugging skills across multiple features/solutions, as evidenced by their experience with various programming languages and tools listed in the resume.
- A closing paragraph that reiterates the candidate's customer-oriented approach, ability to drive business value through collaboration, and enthusiasm for the role.

3. [coverletter], [job description], [resume]
Claude, enhance the provided cover letter by incorporating the following elements from the resume:
- The candidate's experience managing the transition of Wells Fargo's commercial loss models from SAS to PySpark architecture, highlighting their technical expertise and leadership skills.
- Their role in developing seq2seq models using Keras/Pytorch to identify, monitor, and quantify emerging non-financial risks across Wells Fargo's key risk pillars. Explain how this aligns with the job's focus on deploying business solutions.
- The candidate's experience developing a strategic roadmap for SERM that incorporated machine learning model outcomes as overlays/independent variables in traditional econometric models. Tie this to the job's emphasis on driving business value.
- Their work on an LLM framework to identify and monitor emerging risks across regulatory and public media, showcasing their expertise in cutting-edge technologies.
- Adjust the language and tone to match the job description's focus on collaboration, innovation, and customer impact.

4. [coverletter], [job skills]
Claude, revise the cover letter to emphasize the following skills and experiences that align with the job requirements:
- The candidate's ability to lead data-driven projects with business acumen and data science expertise, providing examples from their professional background.
- Their experience applying machine learning solutions and algorithms, preparing and evaluating data, and communicating findings and risks. Use specific projects as evidence.
- The candidate's understanding of the relationship between selected models and business objectives, and their ability to define and design feedback and evaluation methods.
- Their track record of providing feedback, coaching, and support to engineering teams based on business knowledge, technical expertise, and industry trends.
- The candidate's demonstrated coding and debugging skills across multiple features/solutions, highlighting their proficiency in relevant programming languages and tools.
- Their customer-oriented approach and ability to deliver solutions that build trust with Microsoft products.
- Mention of the candidate's advanced degree in a relevant field and 5+ years of data science experience.

5. [coverletter], [job description], [resume]
Claude, update the cover letter to showcase the candidate's fit for the Senior Data Scientist role by incorporating the following:
- An introduction that highlights the candidate's experience working with high-impact engineers and strategic customers to solve complex problems, as mentioned in the job description.
- Examples of the candidate's work on zero-shot information retrieval using retrieval augmented generation, LLM-based classification architectures, and automated validation and testing of data lineage and transformation logic. Explain how these projects demonstrate their ability to develop broadly applicable, high-impact solution patterns.
- Discussion of the candidate's experience with fair lending challenger frameworks and fraudulent check identification using YOLOV and TR-OCR, showcasing their expertise in developing statistical techniques to analyze data and find patterns.
- Mention of the candidate's ability to communicate trends and innovative solutions to stakeholders, as evidenced by their experience presenting results and findings to senior customer stakeholders.
- A closing paragraph that emphasizes the candidate's enthusiasm for working cross-functionally with crews, product teams, and program management to deploy business solutions, as well as their alignment with the team's values of embracing a growth mindset and encouraging diverse viewpoints.